<a href="https://colab.research.google.com/github/PeteJoh/fpl-ai/blob/master/Points_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#------------------------------------------------------
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#  MAKE SURE TO CHANGE GAMEWEEK TO THE GW YOU WANT TO PREDICT
# SAVE TO GITHUB WHEN FINISHED

In [3]:
forward_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

leak_columns = [
    "name",
    "team",
]  # columns that shouldnt be used in training fir fear of data leakage


dropped_columns = [
    "season",
    "opponent",
    "match_result",
   # "position",
    "assists",
    "penalties_missed",
    "bonus",
    "bps",
    "clean_sheets",
    "creativity",
    "goals_conceded",
    "goals_scored",
    "ict_index",
    "influence",
    "own_goals",
    "penalties_saved",
    "red_cards",
    "saves",
    "selected",
    "threat",
    "transfers_balance",
    "transfers_in",
    "transfers_out",
    "yellow_cards",
    "team Goal scored",
    "team Goal conceded"
]  # "value",

midfielder_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

goalkeeper_statistics = ['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'total_points_ex', 'minutes_ex', 'goals_conceded_ex',
       'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'red_cards_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3', 'mean goals_conceded 3',
       'mean ict_index 3',  'mean minutes 3',
       'mean own_goals 3',  'mean penalties_saved 3',
        'mean saves 3',  'mean threat 3',
       'mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

statistics =['value', 'position','was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'yellow_cards_ex','now_cost_ex', 'GW', 'opponent_last_season_position', 'mean assists 3',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3',
       'mean creativity 3', 'mean goals_conceded 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3',
       'mean own_goals 3',
       'mean red_cards 3',  'mean threat 3','mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3','mean saves 3','mean assists all',
       'mean bonus all', 'mean bps all', 'mean clean_sheets all',
       'mean creativity all', 'mean goals_conceded all', 'mean goals_scored all',
       'mean ict_index all', 'mean influence all', 'mean minutes all',
       'mean own_goals all',
       'mean red_cards all',  'mean threat all','mean total_points all',
       'mean value all', 'mean match_result all',
       'mean team Goal scored 3','mean team Goal scored all','mean team Goal conceded 3','mean team Goal conceded all',"ratio_goal_scored all","ratio_goal_scored 3",
       'opp mean team Goal scored 3','opp mean team Goal conceded 3','opp mean team Goal scored all','opp mean team Goal conceded all',"opp mean match_result all"]


date_cols=["day_of week","month","hour","week"]

In [4]:
def convert_minutes(val):
    """CONVERTS MINUTES TO A CATEGORICAL OUTPUT"""
    if val > 10:
        return 1
    else:
        return 0

In [5]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.9 MB/s eta 0:00:00


In [6]:
from hyperopt import tpe,hp,fmin,STATUS_OK,Trials
from hyperopt.pyll.base import scope

In [7]:
import pandas as pd
import numpy as np
import warnings
import os


warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    confusion_matrix,
    accuracy_score,
    f1_score,
)
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,KFold

In [8]:
# ************ CHANGE TO NEXT GW TO BE PREDICTED ******************************
gameweek=14
# *****************************************************************************

In [9]:
# RELOAD THIS DATA FOR EACH MODEL

train = pd.read_csv("/content/gdrive/MyDrive/fplpredict/cleaned_dataset/cleaned_previous_seasons.csv",index_col=0)#/content/drive/MyDrive/Fplpredict /cleaned_previous_seasons.csv", index_col=0)
old_gameweek_cleaned = []
for i in range(1, gameweek):
    old_gameweek_cleaned.append(pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{i}.csv"))
old_gameweeks = pd.concat(old_gameweek_cleaned)[train.columns]
train = pd.concat([train, old_gameweeks])

# data for current gameweek we want to predict on
test = pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{gameweek}.csv", index_col=0)

In [10]:
#CREATE GAMEWEEK PREDICTIONS FOLDER
path = f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}"

# create new single directory
# check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

# data from current season but previous gameweeks

Folder /content/gdrive/MyDrive/fplpredict/predicted_dataset/GW14 already exists


In [11]:
train["position"].value_counts()

,count
position,
MID,53675
DEF,43021
FWD,16008
GK,13166
GKP,1031


In [12]:
train["position"]=train["position"].replace({"GKP":"GK" })

In [13]:
#Predict Points - MAKE SURE TO READ THE DATA IN AGAIN

train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train= train.set_index("index")
#train["date"]=pd.to_datetime(train["kickoff_time"])
train["date"] = pd.to_datetime(train["kickoff_time"], format='ISO8601', errors='coerce')
#train["day_of week"]=train["date"].dt.day_name
train["day_of week"] = train["date"].dt.day_name()  # Added parentheses to call the function
train["month"]=train["date"].dt.month
train["hour"]=train["date"].dt.hour
#train["week"]=train["date"].dt.week
train["week"] = train["date"].dt.isocalendar().week
train.drop(["kickoff_time","date"], axis=1, inplace=True)

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test = test.set_index("index")
#test["date"]=pd.to_datetime(test["kickoff_time"])
test["date"] = pd.to_datetime(test["kickoff_time"], format='ISO8601', errors='coerce')
#test["day_of week"]=test["date"].dt.day_name
test["day_of week"] = test["date"].dt.day_name()  # Added parentheses to call the function
test["month"]=test["date"].dt.month
test["hour"]=test["date"].dt.hour
#test["week"]=test["date"].dt.week
test["week"] = test["date"].dt.isocalendar().week
test.drop(["kickoff_time","date"], axis=1, inplace=True)

target = train[["total_points", "GW","position" ]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
train.drop(dropped_columns, axis=1, inplace=True)
test.drop(dropped_columns, axis=1, inplace=True)


for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name","position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["position"]=train["position"].astype("category")
test["position"]=test["position"].astype("category")
train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

train = train[statistics + leak_columns+date_cols]
test= test[statistics + leak_columns+date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["total_points"],
    test_size=0.1,
    random_state=0,
)

In [14]:
from sklearn.model_selection import KFold

#cross_validator to split the data into folds
folds=KFold(n_splits=8,shuffle=True,random_state=0)

#a dataframe to store the predictions made by each fold
predictions_df=pd.DataFrame()

#list to save the mean absolute errors from validating on each folds
rmse_val=[]
rmse_X=[]

#a simple catboost regressor
model=LGBMRegressor(**{'colsample_bytree': 0.4199299182268318, 'learning_rate': 0.0032874466037521254, 'max_depth': 9, 'min_split_gain': 0.5685369160138952, 'num_leaves': 99, 'reg_alpha': 0.5621526419488447, 'reg_lambda': 0, 'subsample': 0.6534153111773866}, verbose=-50,random_state=0,early_stopping_rounds=200,n_estimators=10000)

#train model, make predictions and check the validation accuracy on  each fold
for i,(train_index,test_index) in enumerate(folds.split(train.drop(leak_columns, axis=1),target["total_points"])):
    train_fold=train.drop(leak_columns, axis=1).iloc[train_index]
    val_fold=train.drop(leak_columns, axis=1).iloc[test_index]
    y_fold=target["total_points"].iloc[train_index]
    y_val_fold=target["total_points"].iloc[test_index]


    model.fit(train_fold,y_fold,eval_set=[(val_fold,y_val_fold)])
    print(i+1)
    prediction=model.predict(test.drop(leak_columns, axis=1))
    predictions_df[i]=prediction
    rmse_val.append(mean_squared_error(model.predict(val_fold),y_val_fold,squared=False))
    rmse_X.append(mean_squared_error(model.predict(train_fold),y_fold,squared=False))
print(rmse_val)
print(rmse_X)

1
2
3
4
5
6
7
8
[1.9947952561223374, 1.9973405343422161, 1.9446485985665256, 1.9341576900546504, 2.0497917016543097, 1.9884697053503728, 1.9578444555909753, 1.9431451651869278]
[1.4108945803239346, 1.7739712311756772, 1.3726122780433407, 1.4099872610934958, 1.557885963087436, 1.4966748224981024, 1.517762454303484, 1.7514085156949384]


In [15]:
print(np.mean(rmse_val))
print(np.mean(rmse_X))
#2.7575293874474336   1.9742746163696216  1.9726217007233091
#2.0174134721607295   1.564712957980843   1.5033778434088017

1.9762741383585394
1.536399638277551


In [16]:
predictions_df[0]

,0
0,0.026576
1,1.140999
2,0.049003
3,0.116689
4,3.248891
...,...
685,0.069881
686,0.106324
687,0.220932
688,0.082017


In [17]:
test["points"]=np.mean(predictions_df, axis=1).values

test[leak_columns + ["points", "value"]].sort_values(
    "points", ascending=False
).to_csv("points.csv")

In [18]:
test[test["position"]=="MID"].sort_values(by="points",ascending=False).head(11)[["name","points","team"]]

,name,points,team
index,,,
Bukayo Saka2024-12-04T20:15:00Z,Bukayo Saka,8.367031,Arsenal
Mohamed Salah2024-12-04T19:30:00Z,Mohamed Salah,8.366251,Liverpool
Cole Palmer2024-12-04T19:30:00Z,Cole Palmer,5.084507,Chelsea
Martin Ødegaard2024-12-04T20:15:00Z,Martin Ødegaard,4.500734,Arsenal
Phil Foden2024-12-04T19:30:00Z,Phil Foden,4.430286,Man City
James Maddison2024-12-05T20:15:00Z,James Maddison,4.031849,Spurs
Mario Lemina2024-12-04T19:30:00Z,Mario Lemina,3.975800,Wolves
Jarrod Bowen2024-12-03T20:15:00Z,Jarrod Bowen,3.930514,West Ham
Brennan Johnson2024-12-05T20:15:00Z,Brennan Johnson,3.829120,Spurs


In [19]:
test[test["position"]=="DEF"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Ashley Young2024-12-04T19:30:00Z,Ashley Young,3.863575,Everton
Virgil van Dijk2024-12-04T19:30:00Z,Virgil van Dijk,3.654668,Liverpool
Andrew Robertson2024-12-04T19:30:00Z,Andrew Robertson,3.623654,Liverpool
Marc Guéhi2024-12-03T19:30:00Z,Marc Guéhi,3.503676,Crystal Palace
Leif Davis2024-12-03T19:30:00Z,Leif Davis,3.376718,Ipswich
Antonee Robinson2024-12-05T19:30:00Z,Antonee Robinson,3.368074,Fulham
James Tarkowski2024-12-04T19:30:00Z,James Tarkowski,3.334793,Everton
Jurriën Timber2024-12-04T20:15:00Z,Jurriën Timber,3.281107,Arsenal
Daniel Muñoz2024-12-03T19:30:00Z,Daniel Muñoz,3.271621,Crystal Palace


In [20]:
test[test["position"]=="GKP"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Jordan Pickford2024-12-04T19:30:00Z,Jordan Pickford,4.231316,Everton
Arijanet Muric2024-12-03T19:30:00Z,Arijanet Muric,3.733924,Ipswich
David Raya Martin2024-12-04T20:15:00Z,David Raya Martin,3.457888,Arsenal
Robert Sánchez2024-12-04T19:30:00Z,Robert Sánchez,3.454712,Chelsea
Dean Henderson2024-12-03T19:30:00Z,Dean Henderson,3.116659,Crystal Palace
José Malheiro de Sá2024-12-04T19:30:00Z,José Malheiro de Sá,3.089821,Wolves
Bernd Leno2024-12-05T19:30:00Z,Bernd Leno,3.014083,Fulham
Caoimhin Kelleher2024-12-04T19:30:00Z,Caoimhin Kelleher,2.928634,Liverpool
André Onana2024-12-04T20:15:00Z,André Onana,2.797189,Man Utd


In [21]:
test[test["position"]=="FWD"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Erling Haaland2024-12-04T19:30:00Z,Erling Haaland,6.639181,Man City
Matheus Santos Carneiro Da Cunha2024-12-04T19:30:00Z,Matheus Santos Carneiro Da Cunha,5.536187,Wolves
Ollie Watkins2024-12-04T20:15:00Z,Ollie Watkins,4.737671,Aston Villa
Nicolas Jackson2024-12-04T19:30:00Z,Nicolas Jackson,4.618412,Chelsea
Jean-Philippe Mateta2024-12-03T19:30:00Z,Jean-Philippe Mateta,4.019144,Crystal Palace
Jørgen Strand Larsen2024-12-04T19:30:00Z,Jørgen Strand Larsen,3.956743,Wolves
Iliman Ndiaye2024-12-04T19:30:00Z,Iliman Ndiaye,3.825613,Everton
Chris Wood2024-12-04T19:30:00Z,Chris Wood,3.809525,Nott'm Forest
João Pedro Junqueira de Jesus2024-12-05T19:30:00Z,João Pedro Junqueira de Jesus,3.298369,Brighton


In [22]:
test["points"].sort_values(ascending=False).head(50)

,points
index,
Bukayo Saka2024-12-04T20:15:00Z,8.367031
Mohamed Salah2024-12-04T19:30:00Z,8.366251
Erling Haaland2024-12-04T19:30:00Z,6.639181
Matheus Santos Carneiro Da Cunha2024-12-04T19:30:00Z,5.536187
Cole Palmer2024-12-04T19:30:00Z,5.084507
Ollie Watkins2024-12-04T20:15:00Z,4.737671
Nicolas Jackson2024-12-04T19:30:00Z,4.618412
Martin Ødegaard2024-12-04T20:15:00Z,4.500734
Phil Foden2024-12-04T19:30:00Z,4.430286


In [23]:
feature_importance = pd.DataFrame(
    {"column": x.columns, "imp": model.feature_importances_}
).sort_values(
    "imp", ascending=False
)

In [24]:
feature_importance.head(50)

,column,imp
71,opp mean team Goal scored all,5970
72,opp mean team Goal conceded all,5842
73,opp mean match_result all,5521
29,mean ict_index 3,4465
24,mean bps 3,4390
66,mean team Goal conceded all,4376
64,mean team Goal scored all,4304
4,percent_value,4238
62,mean match_result all,4159
26,mean creativity 3,3988


In [25]:
feature_importance.tail(30)

,column,imp
12,influence_ex,2477
16,ict_index_ex,2426
63,mean team Goal scored 3,2333
76,hour,2294
10,goals_conceded_ex,2279
3,last_season_position,2040
27,mean goals_conceded 3,2013
17,clean_sheets_ex,1688
14,bonus_ex,1671
65,mean team Goal conceded 3,1655


In [26]:
# Save predictions

In [27]:
test[test["position"]=="DEF"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/defenders_points.csv")
test[test["position"]=="GKP"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/goalkeepers_points.csv")
test[test["position"]=="MID"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/midfielders_points.csv")
test[test["position"]=="FWD"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/forwards_points.csv")